### Проміжні результати роботи над комп'ютерним проєктом

Виконала команда 12: Оксана Москв'як, Андрій Кульбаба, Олег Гуцуляк, Олександр Стаднік.

### Код Хаффмана (Huffman Coding)

Кодування Хафммана реалізоване за допомогою структури дерева Хаффмана як основи.

- створено клас об'єкта вершини дерева 
- створено рекурсивну функцію, яка генерує коди для кожного знаку
- створено функцію побудови дерева через об'єднання двох вершин з мінімальними значеннями в 1
- функція encoding - закодовує файл, наданий нам використовуючи при цьому mmap для роботи з файлами(для збереження пам'яті та підтримки різних форматів), будує дерево, словник значень, генерує коди для знаків, зберігає словник(json) знаків, кількість істиної інформації, початкове розширення файлу для декодування.
- функція decoding - розкодовує закодований нами файл відповідно до словника та початкового розширення.

Інші використані структури:
- Черга(deque) - для видалення переглянутого "декодованого" знаку
- defaultdict - у створенні словника, який зберігає кількість використання певного знаку
- bitarray - для збереження результату кодування
- heapq - використовується як структура min-heap - повне бінарне дерево, корінь якого має найменше значення серед усіх його нащадків, це має виконуватись для лівого і правого піддерев, потрібна щоб об'єднати дві вершини мінімального значення в 1 при побудові.

In [1]:
import os

#### Приклад - "Місто", формат txt

початковий розмір файлу

In [ ]:
misto_size = os.stat('./compression/pidmohylnyy-valerian-petrovych-misto76.txt').st_size
misto_size

911282

вбудований zip-архіватор

In [2]:
misto_size_built = os.stat('./compression/pidmohylnyy-valerian-petrovych-misto76.txt.zip').st_size
misto_size_built

277784

Кодування Гаффмана

In [4]:
misto_size_huffman = os.stat('./compressed_misto_huffman.bin').st_size
misto_size_huffman

304038

#### Приклад - великий json file

початковий розмір файлу

In [6]:
json_size = os.stat('./compression/large-file.json').st_size
json_size

26141343

вбудований zip-архіватор

In [7]:
big_json_size = os.stat('./compression/large-file.json.zip').st_size
big_json_size

3845106

Кодування Гаффмана

In [9]:
big_json_huffman_size = os.stat('./compressed_json_huffman.bin').st_size
big_json_huffman_size

17012930

#### Приклад - csv файл з користувачами на 100000 рядків

початковий розмір файлу

In [10]:
csv_size = os.stat('./compression/customers-100000.csv').st_size
csv_size

17317579

вбудований zip-архіватор

In [11]:
customers_100000_size = os.stat('./compression/customers-100000.csv.zip').st_size
customers_100000_size

8197738

Кодування Гаффмана

In [13]:
customers_100000_huffman_size = os.stat('compressed_customers_100000_huffman.bin').st_size
customers_100000_huffman_size

11956603

#### Приклад jpeg file

початковий розмір файлу

In [3]:
sample_jpeg = os.stat('sample-jpeg.jpeg').st_size
sample_jpeg

1375696

вбудований zip-архіватор

In [4]:
sample_jpeg_zip = os.stat('sample-jpeg.jpeg.zip').st_size
sample_jpeg_zip

1369083

Кодування Гаффмана

In [5]:
sample_jpeg = os.stat('compressed_huffman.bin').st_size
sample_jpeg

1375402

#### Приклад .flac

початковий розмір файлу

In [ ]:
sample_flac = os.stat('sample-flac-small.flac').st_size
sample_flac

1986659

вбудований zip-архіватор

In [7]:
sample_flac_zip = os.stat('sample-flac-small.flac.zip').st_size
sample_flac_zip

1979286

Кодування Гаффмана

In [8]:
sample_flac_huffman = os.stat('compressed_huffman.bin').st_size
sample_flac_huffman

1986536

### LZ77

Реалізація алгоритму стиснення LZ77, заснованого на принципі ковзного вікна (sliding window). Алгоритм знаходить повторювані послідовності байтів у нещодавно оброблених даних і замінює їх посиланнями формату (відстань назад, довжина збігу). Байти, що не мають збігів, кодуються як є (літерали).

Основні компоненти:

    Клас LZ77: Інкапсулює всю логіку стиснення та декомпресії. Має конфігурований розмір ковзного вікна (window_size) та фіксований розмір буфера попереднього перегляду (lookahead_buffer_size).
    Функція find_match:
        Здійснює пошук найдовшого збігу для поточної позиції даних у межах ковзного вікна.
        Використовує хеш-таблицю (dict) для швидкого пошуку потенційних кандидатів на збіг (індексує позиції за хешем 3-байтових послідовностей).
        Оптимізація: Включає механізм очищення хеш-таблиці від застарілих записів (позицій, що вийшли за межі ковзного вікна) перед перебором кандидатів, що прискорює пошук у великих файлах. Очищення реалізоване через list comprehension.
    Функція compress_file:
        Виконує процес стиснення файлу.
        Читає вхідний файл за допомогою mmap та використовує memoryview для ефективного доступу до даних без зайвого копіювання.
        Ітерує по даних, викликаючи find_match для пошуку збігів.
        Формує вихідний стиснений потік бітів за допомогою бібліотеки bitarray. Схема кодування:
            Збіг: 1 біт-прапорець (True) + 16 бітів (12 біт відстань, 4 біти довжина).
            Літерал: 1 біт-прапорець (False) + 8 бітів (байт літерала).
        Метадані: Зберігає на початку стисненого файлу інформацію про розширення оригінального файлу (довжину та самі байти розширення), використовуючи модуль struct.
    Функція decompress_file:
        Виконує процес декомпресії.
        Читає метадані (розширення) з початку файлу.
        Читає основний потік стиснених даних у bitarray.
        Декодує бітовий потік, розрізняючи літерали та пари (відстань, довжина) за прапорцем.
        Відтворює оригінальні дані в bytearray, або додаючи літерали, або копіюючи послідовності з уже декодованої частини буфера відповідно до (відстань, довжина).
        Записує відновлені дані у вихідний файл із збереженим оригінальним розширенням.

Інші використані структури/модулі:

    mmap / memoryview: Для ефективної роботи з файлами та доступу до даних в пам'яті.
    dict (хеш-таблиця): Для індексації позицій та прискорення пошуку збігів.
    bitarray: Для гнучкої роботи з бітовими потоками змінної довжини.
    struct: Для пакування/розпакування бінарних метаданих (інформації про розширення).
    List comprehension: Використовується для реалізації механізму очищення хеш-таблиці.

Тестування программи на стиснення файлів проводилося на фйлах (у дургих дужках розміри після стисненян 7 zip):
- biblija.txt (7,81 МБ ) (2205 кб)
- customers-100000.csv (16.5 мб) (8006 кб)
- large-file.json (24,9 мб) (3754 кб)
- harrypotter.txt (6.21 мб) (2354 кб)
- pidmohylnyy-valerian-petrovych-misto76 (889 кб) (270 кб) 


після стисення фалий отримють рощирення .lz77 
- biblij.lz7 (4131 кб)
- customers-100000.lz77 (12862 кб)
- harrypotter.lz77 (3984 кб)
- large-file.lz77 (10057 кб)
- pidmohylnyy-valerian-petrovych-misto76.lz77 (470 кб)


### LZ78

Алгоритм LZ78 реалізований як клас `LZ78Compressor`, що включає методи для стиснення та розпакування файлів і даних.

- Метод `compress(data: bytes)` приймає байтові дані та виконує стиснення за допомогою словника, який будується динамічно. Алгоритм зчитує послідовності байтів, які вже зустрічалися, і замінює їх індексами в словнику разом з наступним байтом. Повертається список пар `(індекс, байт)`.

- Метод `decompress(pairs: list[tuple[int, bytes]])` приймає список пар `(індекс, байт)` та динамічно будує словник для відновлення оригінальних даних. Починаючи з пустого словника, він реконструює кожен фрагмент і об’єднує їх у повні дані. Повертає розпаковані байти.

- Метод `compress_file(input_path: str, output_path: str)` зчитує файл у байтовому форматі, стискає вміст за допомогою `compress`, і зберігає стиснені пари у бінарному файлі. Індекси зберігаються як 4 байти, після чого йде сам байт.

- Метод `decompress_file(input_path: str, output_path: str)` читає бінарний файл, інтерпретує його як послідовність стиснених пар `(індекс, байт)` і відновлює початкові байти за допомогою `decompress`. Відновлені дані записуються у вивідний файл.

Інші використані структури:

- `dict` — використовується як словник для збереження повторних послідовностей під час стиснення та розпакування.

- `bytearray` — застосовується для ефективного зберігання результату під час розпакування.

- `int.to_bytes` і `int.from_bytes` — вбудовані методи Python для перетворення цілих чисел у байти та навпаки, що дозволяє працювати з файлами без потреби в зовнішніх бібліотеках.

Цей підхід дозволяє стискати будь-які типи файлів, оскільки працює з байтами, а не рядками.


#### Приклад - "Місто", формат txt

початковий розмір файлу

In [1]:
import os

misto_size = os.stat('./compression/pidmohylnyy-valerian-petrovych-misto76.txt').st_size
misto_size

911282

вбудований zip-архіватор

In [2]:
misto_size_built = os.stat('./compression/pidmohylnyy-valerian-petrovych-misto76.txt.zip').st_size
misto_size_built

277752

Кодування LZ78

In [3]:
misto_size_lz78 = os.stat('./compression/compressed_misto_lz78.bin').st_size
misto_size_lz78

699485

#### Приклад - великий json file

початковий розмір файлу

In [4]:
json_size = os.stat('./compression/large-file.json').st_size
json_size

26141343

вбудований zip-архіватор

In [5]:
big_json_size = os.stat('./compression/large-file.json.zip').st_size
big_json_size

3845074

Кодування LZ78

In [6]:
big_json_lz78_size = os.stat('./compression/compressed_json_lz78.bin').st_size
big_json_lz78_size

10461282

#### Приклад - csv файл з користувачами на 100000 рядків

початковий розмір файлу

In [7]:
csv_size = os.stat('./compression/customers-100000.csv').st_size
csv_size

17317579

вбудований zip-архіватор

In [8]:
customers_100000_size = os.stat('./compression/customers-100000.csv.zip').st_size
customers_100000_size

8197707

Кодування LZ78

In [9]:
customers_100000_lz78_size = os.stat('./compression/compressed_customers_100000_lz78.bin').st_size
customers_100000_lz78_size

14171645

### LZW

Алгоритм LZW реалізований як клас `LZWCompressor`, що включає методи для стиснення та розпакування файлів.

- Метод `compress` приймає байтові дані та виконує стиснення за допомогою динамічного словника, який будується під час обробки даних.

- Метод `decompress` приймає стиснені дані у вигляді списку чисел і відновлює початкові байти, використовуючи словник, який також динамічно будується.

- Метод `compress_file` дозволяє стискати файли, зчитуючи їх у байтовому форматі, стискаючи дані та зберігаючи результат у двійковому форматі.

- Метод `decompress_file` дозволяє розпаковувати файли, зчитуючи стиснені дані у двійковому форматі, відновлюючи початкові дані та зберігаючи їх у файл.

Інші використані структури:

- `dict` - для зберігання словника під час стиснення та розпакування.

- `bytearray` - для ефективного зберігання результату розпакування.


#### Приклад - "Місто", формат txt

In [1]:
import os

misto_size = os.stat('./compression/pidmohylnyy-valerian-petrovych-misto76.txt').st_size
misto_size

911282

Місто стиснене за допомогою WinRAR в формат zip

In [4]:
misto_size_zip = os.stat('./compression/pidmohylnyy-valerian-petrovych-misto76.zip').st_size
misto_size_zip

273708

Місто стиснене алгоритмом LZW

In [5]:
misto_size_lzw = os.stat('./compression/pidmohylnyy-valerian-petrovych-misto76.bin').st_size
misto_size_lzw

535340

Насправді, файл можна ще більше стиснути, якщо записувати числа в 3 байти.

In [6]:
misto_size_lzw_3_bytes = os.stat('./compression/pidmohylnyy-valerian-petrovych-misto76.bin').st_size
misto_size_lzw_3_bytes

401505

але це буде працювати лише для малих файлів, а 4 байти більш універсальне рішення, яке підходить для більшості файлів за невелику ціну - воно гірше стискає маленькі файли

#### Приклад - великий json file

Початковий розмір файлу

In [7]:
json_size = os.stat('./compression/large-file.json').st_size
json_size

26141343

WinRAR zip

In [8]:
json_zip = os.stat('./compression/large-file.zip').st_size
json_zip

3772708

LZW

In [9]:
json_lzw = os.stat('./compression/large-file.bin').st_size
json_lzw

7906656

#### Приклад - csv файл з користувачами на 100000 рядків

Початковий розмір файлу

In [10]:
csv_size = os.stat('./compression/customers-100000.csv').st_size
csv_size

17317579

WinRAR zip

In [11]:
csv_zip = os.stat('./compression/customers-100000.zip').st_size
csv_zip

8145737

LZW

In [14]:
csv_lzw = os.stat('./compression/customers-100000.bin').st_size
csv_lzw

10838512

### RLE

## Опис алгоритму RLE

Алгоритм RLE реалізований як клас `RLECompressor`, що включає методи для стиснення та розпакування файлів і байтових даних.

- Метод `compress(data: bytes)` приймає послідовність байтів і стискає повторювані символи. Алгоритм знаходить серії однакових байтів та замінює їх парою `(count, byte)`. Наприклад, `AAAA` → `(4, A)`. Повертає список пар `(count: int, byte: bytes)`.

- Метод `decompress(pairs: list[tuple[int, bytes]])` приймає список пар `(count, byte)` та відновлює оригінальну послідовність байтів, повторюючи кожен байт `count` разів. Повертає розпаковані байти.

- Метод `compress_file(input_path: str, output_path: str)` читає вхідний файл у байтовому режимі, стискає вміст за допомогою `compress`, і записує у вихідний `.bin` файл. Кожна пара `(count, byte)` записується як: 1 байт для кількості (max 255) + 1 байт значення.

- Метод `decompress_file(input_path: str, output_path: str)` читає стиснений файл у вигляді послідовності пар `(count, byte)` і відновлює повні байти. Результат записується у файл в оригінальному форматі.

### Інші використані структури:

- `bytearray` — використовується для поступового зберігання результату у пам’яті при декомпресії.

- `int.to_bytes` і `int.from_bytes` — використовуються для кодування числових значень у байти (і навпаки), що забезпечує сумісність із будь-яким типом файлу.

> **Примітка:** Через свою простоту, RLE ефективно стискає лише дані з великою кількістю повторюваних байтів (наприклад, монохромні зображення або послідовності з нулями). У випадках із складними, неповторюваними даними, алгоритм може навіть збільшити розмір файлу.

#### Приклад стиску зображення, формату bmp

початковий розмір файлу

In [2]:
import os

bmp_size = os.stat('./compression/image1.bmp').st_size
bmp_size

75202

вбудований zip-архіватор

In [3]:
bmp_size_built = os.stat('./compression/image1.bmp.zip').st_size
bmp_size_built

8801

Кодування RLE

In [4]:
bmp_size_rle = os.stat('./compression/compressed_image1_rle.bin').st_size
bmp_size_rle

46810